# **Proyecto 2-A MOS** G27

## Integrantes:
- Juan Esteban Lopez Torres - 202020285
- Bastien Quentin Clement Thirion - 202525085
- Matthieu Maxime Viranin - 202525086
- Alejandro Gonzalez Salazar - 201921465


### Problema fundamental :  
 Optimización de rutas multi-vehículo con restricciones de capacidad, autonomía y operación según el contexto logístico asignado.  

### Justificación :  
 Estos elementos permiten representar adecuadamente la realidad logística del proyecto, ya que cualquier operación de transporte:  
Debe atender clientes con demanda,  
Tiene recursos limitados (vehículos, capacidad, autonomía),   
Y busca minimizar sus costos operativos bajo condiciones específicas del entorno.  
 Sin estos componentes, el modelo no capturaría la complejidad real del sistema ni permitiría comparar soluciones entre diferentes grupos.


## (Caso 1: CVRP Estándar)
### Implementación en Pyomo – Modelo Base (Un solo centro de distribución, vehículos homogéneos)

Implementacion del el **Caso 1 (CVRP estándar)** para LogistiCo:

- Un único centro de distribución (CD01).
- Vehículos homogéneos con la misma capacidad.
- Modelo de **Vehicle Routing Problem con Capacidades (CVRP)** en Pyomo.
- Objetivo: **minimizar el costo operativo**, usando una función de costo basada principalmente en la distancia (extensible a la función objetivo unificada del enunciado).
- Al final, se genera un archivo `verificacion_caso1.csv` con el formato requerido por los lineamientos del proyecto:

`VehicleId, DepotId, InitialLoad, RouteSequence, ClientsServed, DemandsSatisfied, TotalDistance, TotalTime, FuelCost`


In [1]:
# Instalación de Pyomo y del solver GLPK (para Colab)
!pip install pyomo -q
!apt-get install -y glpk-utils > /dev/null


from pyomo.environ import (
    ConcreteModel, Set, Param, Var, NonNegativeReals, Binary,
    Constraint, Objective, SolverFactory, value
)
import math
import csv



In [2]:
# IMPLEMENTACIÓN CVRP ESTÁNDAR (CASO 1)
# Proyecto A

from pyomo.environ import *


# 1. DEFINICIÓN DEL MODELO


model = ConcreteModel()


# 2. CONJUNTOS


# Clientes (IDs estandarizados)
model.N = Set(initialize=[
    "C001","C002","C003","C004","C005","C006","C007","C008","C009","C010",
    "C011","C012","C013","C014","C015","C016","C017","C018","C019","C020",
    "C021","C022","C023","C024"
])

# Vehículos
model.V = Set(initialize=[
    "V001","V002","V003","V004","V005","V006","V007","V008"
])

# Depot
depot = "CD01"

# Nodos (depot + clientes)
model.N0 = Set(initialize=[depot] + list(model.N))

# 3. PARÁMETROS


# Demandas
demand_dict = {
    "C001":13,"C002":15,"C003":12,"C004":15,"C005":20,"C006":17,
    "C007":17,"C008":20,"C009":20,"C010":15,"C011":17,"C012":12,
    "C013":21,"C014":15,"C015":17,"C016":10,"C017":25,"C018":12,
    "C019":11,"C020":15,"C021":14,"C022":18,"C023":15,"C024":11
}
model.demand = Param(model.N, initialize=demand_dict)

# Capacidades de vehículos
capacity_dict = {
    "V001":130,"V002":140,"V003":120,"V004":100,
    "V005":70,"V006":55,"V007":110,"V008":114
}
model.capacity = Param(model.V, initialize=capacity_dict)

# Distancia – Parametrizada genéricamente (rellenada desde OR-Tools)
def dist_placeholder(m, i, j):
    return 0.0
model.dist = Param(model.N0, model.N0, initialize=dist_placeholder, within=NonNegativeReals, mutable=True)

# 4. VARIABLES DE DECISIÓN

# x[i,j,v] = 1 si el vehículo v viaja de i → j
model.x = Var(model.N0, model.N0, model.V, domain=Binary)

# Variables MTZ (subtour elimination)
model.u = Var(model.N, model.V, domain=NonNegativeReals)

# 5. FUNCIÓN OBJETIVO
# Minimización de la distancia total

def objective_rule(m):
    return sum(m.dist[i,j] * m.x[i,j,v] for i in m.N0 for j in m.N0 for v in m.V)
model.OBJ = Objective(rule=objective_rule, sense=minimize)

# 6. RESTRICCIONES DEL CVRP

# No permitir arcos i → i
def nolook_rule(m, i, v):
    return m.x[i,i,v] == 0
model.NoLoops = Constraint(model.N0, model.V, rule=nolook_rule)

# Cada cliente es visitado exactamente una vez
def visit_once_rule(m, j):
    return sum(m.x[i,j,v] for i in m.N0 for v in m.V if i != j) == 1
model.VisitOnce = Constraint(model.N, rule=visit_once_rule)

# Flujo de entrada = flujo de salida para cada cliente y vehículo
def flow_balance_rule(m, h, v):
    return sum(m.x[i,h,v] for i in m.N0 if i != h) == \
           sum(m.x[h,j,v] for j in m.N0 if j != h)
model.FlowBalance = Constraint(model.N, model.V, rule=flow_balance_rule)

# Un vehículo sale del depot a lo sumo una vez
def depot_out_rule(m, v):
    return sum(m.x[depot, j, v] for j in m.N) <= 1
model.DepotOut = Constraint(model.V, rule=depot_out_rule)

# Un vehículo regresa al depot a lo sumo una vez
def depot_in_rule(m, v):
    return sum(m.x[i, depot, v] for i in m.N) <= 1
model.DepotIn = Constraint(model.V, rule=depot_in_rule)

# Balance depot: si sale, debe volver
def depot_balance_rule(m, v):
    return sum(m.x[depot,j,v] for j in m.N) == \
           sum(m.x[i,depot,v] for i in m.N)
model.DepotBalance = Constraint(model.V, rule=depot_balance_rule)

# MTZ: Subtour elimination (solo para i,j clientes)
def mtz_rule(m, i, j, v):
    if i == j:
        return Constraint.Skip
    return m.u[j,v] >= m.u[i,v] + m.demand[j] - m.capacity[v] * (1 - m.x[i,j,v])
model.MTZ = Constraint(model.N, model.N, model.V, rule=mtz_rule)

# Límites de carga u[i,v]
def lower_u(m, i, v):
    return m.u[i,v] >= m.demand[i]
model.ULower = Constraint(model.N, model.V, rule=lower_u)

def upper_u(m, i, v):
    return m.u[i,v] <= m.capacity[v]
model.UUpper = Constraint(model.N, model.V, rule=upper_u)



In [3]:
# ============================================
#  Configuración de rutas para Caso 1 — CSV locales
# ============================================

import os
import pandas as pd
import math
import csv
import folium
from IPython.display import display
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Carpeta donde subiste los CSV
DATA_DIR = "/content/datos_caso1"

# Crear carpeta si no existe
os.makedirs(DATA_DIR, exist_ok=True)

# Paths
CLIENTS_CSV  = f"{DATA_DIR}/clients.csv"
DEPOTS_CSV   = f"{DATA_DIR}/depots.csv"
VEHICLES_CSV = f"{DATA_DIR}/vehicles.csv"
PARAM_CSV    = f"{DATA_DIR}/parameters_base.csv"

print(" Leyendo archivos desde:", DATA_DIR)

# =============================
# 1. Lectura
# =============================

clients_df  = pd.read_csv(CLIENTS_CSV)
depots_df   = pd.read_csv(DEPOTS_CSV)
vehicles_df = pd.read_csv(VEHICLES_CSV)
params_df   = pd.read_csv(PARAM_CSV)

print(" CSV cargados correctamente.")

# =============================
# 2. Estructura de nodos
# =============================

# Depot único
depot = depots_df.iloc[0]
depot_id  = depot["StandardizedID"]
depot_lat = depot["Latitude"]
depot_lon = depot["Longitude"]

# Clientes
clients_data = [
    (row["StandardizedID"], row["Latitude"], row["Longitude"], row["Demand"])
    for _, row in clients_df.iterrows()
]

# Vehículos
vehicles_data = [
    (row["StandardizedID"], row["Capacity"], row["Range"])
    for _, row in vehicles_df.iterrows()
]

# Parámetros base
param = {r["Parameter"]: r["Value"] for _, r in params_df.iterrows()}
fuel_price     = float(param.get("fuel_price", 16300))
fuel_efficiency = float(param.get("fuel_efficiency", 30))
speed_kmh       = float(param.get("speed_kmh", 30))

print(" Parámetros cargados.")

# =============================
# 3. Preparación modelo
# =============================

node_ids = [depot_id] + [c[0] for c in clients_data]

lat = {depot_id: depot_lat}
lon = {depot_id: depot_lon}
demand = {}

for cid, la, lo, dem in clients_data:
    lat[cid] = la
    lon[cid] = lo
    demand[cid] = dem

# --- Haversine ---
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = (
        math.sin(dphi/2)**2 +
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    )
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

num_nodes = len(node_ids)
distance_km = [[0]*num_nodes for _ in range(num_nodes)]

for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            ni = node_ids[i]
            nj = node_ids[j]
            distance_km[i][j] = haversine_km(lat[ni], lon[ni], lat[nj], lon[nj])

distance_m = [[int(distance_km[i][j]*1000) for j in range(num_nodes)] for i in range(num_nodes)]

demands = [0] + [demand[c] for c in node_ids[1:]]
vehicle_ids = [v[0] for v in vehicles_data]
vehicle_capacities = [v[1] for v in vehicles_data]
num_vehicles = len(vehicles_data)
depot_index = 0

# =============================
# 4. OR-Tools VRP
# =============================

manager = pywrapcp.RoutingIndexManager(num_nodes, num_vehicles, depot_index)
routing = pywrapcp.RoutingModel(manager)

def distance_cb(from_idx, to_idx):
    a = manager.IndexToNode(from_idx)
    b = manager.IndexToNode(to_idx)
    return distance_m[a][b]

transit_idx = routing.RegisterTransitCallback(distance_cb)
routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)

def demand_cb(idx):
    return demands[manager.IndexToNode(idx)]

demand_idx = routing.RegisterUnaryTransitCallback(demand_cb)

routing.AddDimensionWithVehicleCapacity(
    demand_idx,
    0,
    vehicle_capacities,
    True,
    "Capacity"
)

params = pywrapcp.DefaultRoutingSearchParameters()
params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
params.time_limit.FromSeconds(30)

solution = routing.SolveWithParameters(params)

if not solution:
    print(" No se encontró solución.")
else:
    print(" Solución encontrada.")


 Leyendo archivos desde: /content/datos_caso1
 CSV cargados correctamente.
 Parámetros cargados.
 Solución encontrada.


In [4]:
OUTPUT_FILE = "/content/verificacion_caso1.csv"

if solution:
    with open(OUTPUT_FILE, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow([
            "VehicleId","DepotId","InitialLoad","RouteSequence",
            "ClientsServed","DemandsSatisfied","TotalDistance",
            "TotalTime","FuelCost"
        ])

        for v in range(num_vehicles):
            idx = routing.Start(v)
            route = []
            while not routing.IsEnd(idx):
                n = manager.IndexToNode(idx)
                route.append(node_ids[n])
                idx = solution.Value(routing.NextVar(idx))
            route.append(node_ids[manager.IndexToNode(idx)])

            served = [n for n in route if n != depot_id]
            if not served:
                continue

            dem_vals = [demand[n] for n in served]
            initial_load = sum(dem_vals)

            dist = 0
            for i in range(len(route)-1):
                a = node_ids.index(route[i])
                b = node_ids.index(route[i+1])
                dist += distance_km[a][b]

            time_min = (dist/speed_kmh)*60
            gallons = dist / fuel_efficiency
            cost = gallons * fuel_price

            w.writerow([
                vehicle_ids[v],
                depot_id,
                round(initial_load,2),
                "-".join(route),
                len(served),
                "-".join(str(x) for x in dem_vals),
                round(dist,4),
                round(time_min,2),
                round(cost,2)
            ])

    print(" Archivo generado:", OUTPUT_FILE)


 Archivo generado: /content/verificacion_caso1.csv


In [5]:
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=12)

folium.Marker([depot_lat, depot_lon], popup=f"Depot {depot_id}",
              icon=folium.Icon(color="red")).add_to(m)

for cid, la, lo, dem in clients_data:
    folium.CircleMarker([la, lo], radius=4,
                         popup=f"{cid} - Demanda {dem}",
                         color="blue", fill=True).add_to(m)

colors = ["blue","green","purple","orange","black","darkred","cadetblue"]

for v in range(num_vehicles):
    idx = routing.Start(v)
    coords = []
    while not routing.IsEnd(idx):
        n = manager.IndexToNode(idx)
        coords.append((lat[node_ids[n]], lon[node_ids[n]]))
        idx = solution.Value(routing.NextVar(idx))
    coords.append((lat[node_ids[manager.IndexToNode(idx)]],
                   lon[node_ids[manager.IndexToNode(idx)]]))
    if len(coords)>2:
        folium.PolyLine(coords, color=colors[v%len(colors)], weight=3).add_to(m)

display(m)


La configuración inicial del modelo de ruteo permitió implementar y resolver el Problema de Rutas de Vehículos con capacidad (CVRP) en un entorno urbano realista dentro de Bogotá. A partir de los datos georreferenciados de 24 clientes y un centro de distribución, el sistema de optimización  generó rutas factibles respetando las capacidades máximas de la flota y minimizando la distancia total recorrida.

Los resultados obtenidos son los siguientes:

- **Archivo de validación generado:** `verificacion_caso1.csv`  
  → Este archivo permite rastrear:
  - Clientes atendidos por cada vehículo  
  - Demandas satisfechas  
  - Secuencia completa de la ruta (CD → clientes → CD)  
  - Distancia real por vehículo  
  - Tiempo estimado  
  - Costo de combustible

La metodología siguió los lineamientos establecidos para el Caso Base:

1. **Uso de distancias reales (Haversine):**  
2. **Restricciones de capacidad:**  
3. **Minimización del recorrido total:**  
4. **Visualización geográfica:**  


En resumen, el modelo logró planificar rutas factibles, minimizando la distancia y cumpliendo las restricciones operativas básicas, lo cual sienta las bases para las siguientes fases del proyecto donde se incorporarán centros múltiples, ventanas de tiempo y condiciones urbanas específicas.


## Caso 2: Logística Urbana con Múltiples Depósitos (MDVRP)

### Evolución del Modelo
A diferencia del Caso Base (CVRP estándar con un único origen), este escenario aborda la complejidad real de la operación en Bogotá introduciendo la gestión de **Múltiples Centros de Distribución (CDs)**.

El desafío principal deja de ser solo el ruteo y pasa a ser la **asignación estratégica**: decidir qué vehículo sale de qué depósito para minimizar costos sin violar las restricciones de inventario.

### Enfoque de Resolución
Para cumplir con los requisitos de implementación formal, desarrollamos la solución a **Pyomo** utilizando un enfoque de **Programación Lineal Entera Mixta (MILP)** con las siguientes características avanzadas:

1.  **Modelo MDVRP (Multi-Depot VRP):** Se introducen variables binarias de decisión $z_{k,d}$ para asignar dinámicamente cada vehículo a un depósito óptimo.
2.  **Restricciones de Capacidad Robustas:** Se modela el inventario de cada depósito. Para evitar la infactibilidad matemática en casos donde la demanda local supera la capacidad instalada (como se advierte en el *README*), se implementan **variables de holgura (slack)** con penalización en la función objetivo. Esto permite al modelo reportar déficits estructurales en lugar de fallar.
3.  **Solver Exacto:** Se utiliza **CBC (Coin-OR)** con un tiempo extendido de búsqueda para garantizar la convergencia al óptimo global o la mejor solución factible posible.

---

In [6]:
# ==========================================
# CELDA 1: INSTALACIÓN DE DEPENDENCIAS
# ==========================================
import sys
import os

# Detectar si estamos en Google Colab o Linux para instalar CBC
if 'google.colab' in sys.modules or os.name == 'posix':
    print("Instalando COIN-OR CBC y librerías necesarias...")
    !apt-get install -y -qq coinor-cbc
    !pip install -q pyomo pandas folium matplotlib
else:
    print("Estás en Windows/Mac local. Asegúrate de tener el binario 'cbc' en tu PATH.")

print("Entorno configurado con CBC.")

Instalando COIN-OR CBC y librerías necesarias...
Entorno configurado con CBC.


In [7]:
# ==========================================
# CELDA 2: IMPORTACIÓN Y PROCESAMIENTO DINÁMICO DE DATOS
# ==========================================
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import folium
import math
from IPython.display import display

# Carpeta donde subiste los CSV
DATA_DIR = "/content/datos_caso2"

# Crear carpeta si no existe
os.makedirs(DATA_DIR, exist_ok=True)

# Paths
CLIENTS_CSV  = f"{DATA_DIR}/clients.csv"
DEPOTS_CSV   = f"{DATA_DIR}/depots.csv"
VEHICLES_CSV = f"{DATA_DIR}/vehicles.csv"
PARAM_CSV    = f"{DATA_DIR}/parameters_urban.csv"

print(" Leyendo archivos desde:", DATA_DIR)

# 1. LEER ARCHIVOS
try:
    df_clients = pd.read_csv(CLIENTS_CSV)
    df_depots = pd.read_csv(DEPOTS_CSV)
    df_vehicles = pd.read_csv(VEHICLES_CSV)
    df_params = pd.read_csv(PARAM_CSV)
except FileNotFoundError as e:
    print(f" Error: No se encontró el archivo {e.filename}. Asegúrate de subirlos.")
    raise

# --- 2. PROCESAR PARÁMETROS ECONÓMICOS ---
# Convertir el CSV de parámetros en un diccionario clave-valor para fácil acceso
param_dict = dict(zip(df_params['Parameter'], df_params['Value']))

PARAMS = {
    'COST_FIXED': float(param_dict.get('C_fixed', 50000)),
    'COST_DIST': float(param_dict.get('C_dist', 2500)),
    'COST_TIME': float(param_dict.get('C_time', 7600)),
    'FUEL_PRICE': float(param_dict.get('fuel_price', 16300)),
    'AVG_SPEED': 25 # Valor estándar del README (generalmente no está en el csv)
}

# Calcular Eficiencias Promedio (Min + Max) / 2 dinámicamente
def get_avg_eff(v_type):
    try:
        min_v = float(param_dict.get(f'fuel_efficiency_{v_type}_min', 30))
        max_v = float(param_dict.get(f'fuel_efficiency_{v_type}_max', 30))
        return (min_v + max_v) / 2
    except:
        return 30.0

EFFICIENCY = {
    'small van': get_avg_eff('van_small'),
    'medium van': get_avg_eff('van_medium'),
    'light truck': get_avg_eff('truck_light')
}

# --- 3. PROCESAR CLIENTES ---
clients_data = []
for _, row in df_clients.iterrows():
    clients_data.append({
        'id': str(row['StandardizedID']).strip(),
        'lat': float(row['Latitude']),
        'lon': float(row['Longitude']),
        'dem': float(row['Demand'])
    })

# --- 4. PROCESAR DEPÓSITOS ---
depots_data = []
for _, row in df_depots.iterrows():
    depots_data.append({
        'id': str(row['StandardizedID']).strip(),
        'lat': float(row['Latitude']),
        'lon': float(row['Longitude']),
        'cap': float(row['Capacity'])
    })

# --- 5. PROCESAR VEHÍCULOS (CON CORRECCIÓN DE ERRORES) ---
# Detectar si las columnas están invertidas (Error conocido en los datos)
# Si la columna 'StandardizedID' contiene "van" o "truck", está mal.
first_val = str(df_vehicles.iloc[0]['StandardizedID'])
col_id = 'StandardizedID'
col_type = 'VehicleType'

if 'van' in first_val.lower() or 'truck' in first_val.lower():
    print(" Advertencia: Columnas invertidas detectadas en vehicles.csv. Corrigiendo al vuelo...")
    col_id = 'VehicleType'     # El ID real está en la columna Type
    col_type = 'StandardizedID' # El Tipo real está en la columna ID

vehicles_data = []
for _, row in df_vehicles.iterrows():
    v_type = str(row[col_type]).strip().lower() # Normalizar a minúsculas

    # Normalizar nombres de tipos para que coincidan con EFFICIENCY
    if 'small' in v_type: v_type = 'small van'
    elif 'medium' in v_type: v_type = 'medium van'
    elif 'light' in v_type: v_type = 'light truck'

    vehicles_data.append({
        'id': str(row[col_id]).strip(),
        'type': v_type,
        'cap': float(row['Capacity']),
        'range': float(row['Range'])
    })

# --- 6. ESTRUCTURACIÓN FINAL (NODOS Y LISTAS) ---
nodes = {}
# Mapear Depósitos
for d in depots_data:
    nodes[d['id']] = {'lat': d['lat'], 'lon': d['lon'], 'cap': d['cap'], 'dem': 0, 'type': 'depot'}
# Mapear Clientes
for c in clients_data:
    nodes[c['id']] = {'lat': c['lat'], 'lon': c['lon'], 'dem': c['dem'], 'cap': 0, 'type': 'client'}

# Diccionario de vehículos
vehicles = {v['id']: v for v in vehicles_data}

# Listas de IDs para Sets de Pyomo
all_nodes = [d['id'] for d in depots_data] + [c['id'] for c in clients_data]
client_ids = [c['id'] for c in clients_data]
depot_ids = [d['id'] for d in depots_data]
vehicle_ids = [v['id'] for v in vehicles_data]

print("\n Datos procesados y cargados correctamente en memoria.")
print(f"   - Clientes:  {len(clients_data)}")
print(f"   - Depósitos: {len(depots_data)}")
print(f"   - Vehículos: {len(vehicles_data)}")
print(f"   - Costos leídos: Fijo=${PARAMS['COST_FIXED']:,}, Combustible=${PARAMS['FUEL_PRICE']:,}/gal")
print(f"   - Eficiencias calculadas: {EFFICIENCY}")

 Leyendo archivos desde: /content/datos_caso2
 Advertencia: Columnas invertidas detectadas en vehicles.csv. Corrigiendo al vuelo...

 Datos procesados y cargados correctamente en memoria.
   - Clientes:  9
   - Depósitos: 12
   - Vehículos: 6
   - Costos leídos: Fijo=$50,000.0, Combustible=$16,300.0/gal
   - Eficiencias calculadas: {'small van': 40.0, 'medium van': 30.0, 'light truck': 25.0}


### Formulación Matemática y Estrategia de Robustez

Para garantizar la operatividad del modelo ante restricciones de inventario estrictas, se implementó una formulación **robusta** que penaliza pero permite violaciones controladas de capacidad.

**Función Objetivo Extendida:**
El objetivo es minimizar el Costo Total Generalizado ($Z$), compuesto por costos operativos reales y penalizaciones virtuales:

$$\min Z = \sum (C_{fijo} + C_{dist} + C_{tiempo} + C_{combustible}) + \sum_{d \in D} (Slack_d \times P)$$

Donde:
* **Costos Operativos:** Suma de activación de flota, distancia recorrida, tiempo de conductor y consumo de gasolina según eficiencia del vehículo.
* **$Slack_d$:** Variable continua no negativa que representa el exceso de carga (kg) en el depósito $d$.
* **$P$:** Penalización ("Big M"), configurada en $10,000,000$ COP/kg.

**Justificación:**
Esta estructura asegura que el solver priorice siempre el cumplimiento de capacidades ($Slack = 0$). Solo en casos donde la demanda local supere físicamente la capacidad instalada (infactibilidad estructural), el modelo activará la variable de holgura, permitiendo identificar y cuantificar los **cuellos de botella** en la infraestructura.

In [8]:
# ==========================================
# CELDA 3: MODELO PYOMO OPTIMIZADO
# ==========================================

# 1. Distancias (Haversine)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat, dlon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

dist_matrix = {}
time_matrix = {}
for i in all_nodes:
    for j in all_nodes:
        d = haversine(nodes[i]['lat'], nodes[i]['lon'], nodes[j]['lat'], nodes[j]['lon'])
        dist_matrix[i, j] = d
        time_matrix[i, j] = d / PARAMS['AVG_SPEED']

# 2. Inicializar Modelo
m = pyo.ConcreteModel()
m.V = pyo.Set(initialize=vehicle_ids)
m.N = pyo.Set(initialize=client_ids)
m.D = pyo.Set(initialize=depot_ids)
m.Nodes = pyo.Set(initialize=all_nodes)

# Variables
m.x = pyo.Var(m.Nodes, m.Nodes, m.V, domain=pyo.Binary)
m.z = pyo.Var(m.V, m.D, domain=pyo.Binary)
m.u = pyo.Var(m.N, m.V, domain=pyo.NonNegativeReals)
m.q = pyo.Var(m.V, m.D, domain=pyo.NonNegativeReals) # Carga asignada
m.slack_depot = pyo.Var(m.D, domain=pyo.NonNegativeReals) # Holgura de capacidad

# Función Objetivo (Con Penalización de 10 Millones para forzar cumplimiento)
PENALTY = 10000000

def obj_rule(model):
    c_dist = sum(dist_matrix[i,j] * model.x[i,j,k] * PARAMS['COST_DIST'] for i in model.Nodes for j in model.Nodes for k in model.V)
    c_time = sum(time_matrix[i,j] * model.x[i,j,k] * PARAMS['COST_TIME'] for i in model.Nodes for j in model.Nodes for k in model.V)
    c_fixed = sum(model.z[k,d] * PARAMS['COST_FIXED'] for k in model.V for d in model.D)
    c_fuel = 0
    for k in model.V:
        eff = EFFICIENCY.get(vehicles[k]['type'], 30.0)
        c_fuel += sum(dist_matrix[i,j] * model.x[i,j,k] for i in model.Nodes for j in model.Nodes) / eff * PARAMS['FUEL_PRICE']

    # Penalización por uso de holgura
    c_penalty = sum(model.slack_depot[d] for d in model.D) * PENALTY
    return c_dist + c_time + c_fixed + c_fuel + c_penalty

m.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Restricciones
m.Visit = pyo.Constraint(m.N, rule=lambda mod, i: sum(mod.x[i,j,k] for j in mod.Nodes for k in mod.V if i!=j) == 1)
m.Flow = pyo.Constraint(m.N, m.V, rule=lambda mod, h, k: sum(mod.x[i,h,k] for i in mod.Nodes if i!=h) == sum(mod.x[h,j,k] for j in mod.Nodes if j!=h))
m.OneDepot = pyo.Constraint(m.V, rule=lambda mod, k: sum(mod.z[k,d] for d in mod.D) <= 1)
m.StartAtDepot = pyo.Constraint(m.V, m.D, rule=lambda mod, k, d: sum(mod.x[d,j,k] for j in mod.N) == mod.z[k,d])
m.EndAtDepot = pyo.Constraint(m.V, m.D, rule=lambda mod, k, d: sum(mod.x[i,d,k] for i in mod.N) == mod.z[k,d])
m.CapVehicle = pyo.Constraint(m.V, rule=lambda mod, k: sum(nodes[i]['dem'] * sum(mod.x[i,j,k] for j in mod.Nodes if i!=j) for i in mod.N) <= vehicles[k]['cap'])
m.RangeVehicle = pyo.Constraint(m.V, rule=lambda mod, k: sum(dist_matrix[i,j] * mod.x[i,j,k] for i in mod.Nodes for j in mod.Nodes) <= vehicles[k]['range'])

# Restricciones de Capacidad de Depósito
m.LoadBalance = pyo.Constraint(m.V, rule=lambda mod, k: sum(mod.q[k,d] for d in mod.D) == sum(nodes[i]['dem'] * sum(mod.x[i,j,k] for j in mod.Nodes if i!=j) for i in mod.N))
m.LoadUpperBound = pyo.Constraint(m.V, m.D, rule=lambda mod, k, d: mod.q[k,d] <= vehicles[k]['cap'] * mod.z[k,d])
m.DepotCap = pyo.Constraint(m.D, rule=lambda mod, d: sum(mod.q[k,d] for k in mod.V) <= nodes[d]['cap'] + mod.slack_depot[d])
m.MTZ = pyo.Constraint(m.N, m.N, m.V, rule=lambda mod, i, j, k: mod.u[i,k] - mod.u[j,k] + len(mod.N)*mod.x[i,j,k] <= len(mod.N)-1 if i!=j else pyo.Constraint.Skip)

print("Modelo construido.")

Modelo construido.


In [9]:
# ==========================================
# CELDA 4: SOLVER CBC Y GENERACIÓN DE DATOS
# ==========================================
import pandas as pd

# 1. Configuración y Resolución
solver = SolverFactory('cbc')
solver.options['seconds'] = 600

print(f" Optimizando con CBC (Max {solver.options['seconds']}s)...")
results = solver.solve(m)

term_cond = results.solver.termination_condition
print(f"\n--- ESTADO: {term_cond} ---")

# 2. Procesamiento de Resultados
if term_cond in [TerminationCondition.optimal, TerminationCondition.maxTimeLimit, TerminationCondition.feasible]:
    output_data = []

    for k in m.V:
        # A. Encontrar depósito de inicio
        used_depot = None
        for d in m.D:
            if pyo.value(m.z[k,d]) > 0.5:
                used_depot = d
                break

        if used_depot:
            # B. Reconstrucción de Ruta
            route = [used_depot]
            curr = used_depot
            dist_total = 0

            # Límite de seguridad para evitar bucles infinitos
            for step in range(len(all_nodes) + 5):
                next_node = None

                # Buscar el siguiente nodo conectado (x=1)
                for j in m.Nodes:
                    if j != curr:
                        if pyo.value(m.x[curr, j, k]) > 0.5:
                            next_node = j
                            break

                # Si encontramos el siguiente nodo:
                if next_node:
                    dist_segment = dist_matrix[curr, next_node]
                    dist_total += dist_segment
                    route.append(next_node)
                    curr = next_node

                    # CONDICIÓN DE PARADA: Si volvimos a un depósito
                    if curr in depot_ids:
                        break
                else:
                    # Ruta incompleta o rota
                    break

            # C. Calcular Métricas
            # Filtrar solo clientes para sumar demanda
            client_nodes = [n for n in route if n in client_ids]
            initial_load = sum(nodes[n]['dem'] for n in client_nodes)

            # Eficiencia
            v_type = vehicles[k]['type']
            eff = EFFICIENCY.get(v_type, 30.0)

            # Costos
            fuel_cost = (dist_total / eff) * PARAMS['FUEL_PRICE']
            time_total = (dist_total / PARAMS['AVG_SPEED']) * 60 # Minutos

            output_data.append({
                'VehicleId': k,
                'DepotId': used_depot,
                'InitialLoad': initial_load,
                'RouteSequence': "-".join(route),
                'ClientsServed': len(client_nodes),
                'DemandsSatisfied': "-".join([str(int(nodes[n]['dem'])) for n in client_nodes]),
                'TotalDistance': round(dist_total, 2),
                'TotalTime': round(time_total, 2),
                'FuelCost': round(fuel_cost, 0)
            })

    # 3. Generar DataFrame y CSV
    df_out = pd.DataFrame(output_data)

    # Asegurar que las columnas numéricas sean reconocidas como números
    cols_num = ['TotalDistance', 'TotalTime', 'FuelCost', 'InitialLoad', 'ClientsServed']
    for c in cols_num:
        if c in df_out.columns:
            df_out[c] = pd.to_numeric(df_out[c])

    df_out.to_csv('verificacion_caso2.csv', index=False)
    print(" Solución procesada. Archivo 'verificacion_caso2.csv' generado correctamente.")

    # Mostrar vista previa para verificar que no haya ceros
    try:
        from IPython.display import display
        display(df_out.head())
    except:
        print(df_out.head())

else:
    print(" No se encontró solución.")

 Optimizando con CBC (Max 600s)...

--- ESTADO: optimal ---
 Solución procesada. Archivo 'verificacion_caso2.csv' generado correctamente.


,VehicleId,DepotId,InitialLoad,RouteSequence,ClientsServed,DemandsSatisfied,TotalDistance,TotalTime,FuelCost
0,V001,CD09,42.0,CD09-C008-C007-C005-C003-CD09,4,10-12-5-15,21.76,52.23,8868.0
1,V002,CD05,26.0,CD05-C006-C002-CD05,2,11-15,15.63,37.52,8495.0
2,V003,CD12,18.0,CD12-C001-C004-CD12,2,12-6,21.00,50.39,11408.0
3,V005,CD11,15.0,CD11-C009-CD11,1,15,6.88,16.51,3737.0


### Análisis de Resultados y Visualización Geoespacial

A continuación, se procesan los resultados de la optimización para generar inteligencia de negocio accionable.

**Estructura del Análisis:**
1.  **KPIs Financieros:** Se desglosa el costo en sus componentes reales (dinero a desembolsar) vs. costos matemáticos (multas).
    * *Nota:* Si el "Costo Matemático" es superior al "Real", indica que existen depósitos operando por encima de su capacidad nominal.
2.  **Estado de la Infraestructura:** Tabla detallada de utilización de cada Centro de Distribución (CD).
    * 🟢 **OK:** Operación normal.
    * 🔴 **SOBRECUPO:** El depósito excede su capacidad (cuello de botella identificado).
    * ⚪ **SIN USO:** Depósito no asignado por ineficiencia o lejanía.
3.  **Visualización de Rutas (Folium):**
    * Las rutas se colorean según el **Depósito de Origen** para validar visualmente la clusterización geográfica.
    * Los marcadores de depósito incluyen alertas visuales si presentan sobrecupo.

In [10]:
# ==========================================
# CELDA 5: REPORTE GERENCIAL
# ==========================================
if 'df_out' in locals() and not df_out.empty:
    print("\n" + "="*60)
    print("          REPORTE - CASO 2 (CBC)")
    print("="*60)

    # Asegurar tipos numéricos
    df_out['TotalDistance'] = pd.to_numeric(df_out['TotalDistance'])
    df_out['TotalTime'] = pd.to_numeric(df_out['TotalTime'])
    df_out['FuelCost'] = pd.to_numeric(df_out['FuelCost'])

    # Cálculo de Costos
    total_dist = df_out['TotalDistance'].sum()
    cost_dist = total_dist * PARAMS['COST_DIST']
    cost_time = (df_out['TotalTime'].sum() / 60) * PARAMS['COST_TIME']
    cost_fuel = df_out['FuelCost'].sum()
    cost_fixed = len(df_out) * PARAMS['COST_FIXED']

    total_operation = cost_dist + cost_time + cost_fuel + cost_fixed

    # Penalizaciones Teóricas
    slack_cost = 0
    if hasattr(m, 'slack_depot'):
        slack_cost = sum(pyo.value(m.slack_depot[d]) for d in m.D) * PENALTY

    print(f"1. RESUMEN FINANCIERO")
    print(f"   ------------------------------------------------")
    print(f"   COSTO OPERATIVO REAL:      ${total_operation:,.0f} COP")
    if slack_cost > 0.1: # Umbral bajo por si hay decimales
        print(f"   COSTO MATEMÁTICO (+MULTA): ${total_operation + slack_cost:,.0f} COP")
    print(f"   ------------------------------------------------")
    print(f"   - Flota (Fijo):            ${cost_fixed:,.0f} ({len(df_out)} vehículos)")
    print(f"   - Distancia Variable:      ${cost_dist:,.0f} ({total_dist:.1f} km)")
    print(f"   - Tiempo:                  ${cost_time:,.0f}")
    print(f"   - Combustible:             ${cost_fuel:,.0f}")

    print(f"\n2. ESTADO DE LOS DEPÓSITOS")
    print(f"   ------------------------------------------------")
    print(f"   {'Depósito':<10} | {'Capacidad':<10} | {'Uso':<10} | {'Estado':<15}")

    for d_id in depot_ids:
        usage = sum(pyo.value(m.q[k,d_id]) for k in m.V)
        cap = nodes[d_id]['cap']
        slack = pyo.value(m.slack_depot[d_id])

        status = "🟢 OK"
        if slack > 0.1: status = f"🔴 SOBRECUPO (+{slack:.1f})"
        elif usage == 0: status = "⚪ SIN USO"

        if usage > 0 or slack > 0.1:
            print(f"   {d_id:<10} | {cap:<10.0f} | {usage:<10.0f} | {status:<15}")
    print("="*60)
else:
    print(" No hay datos para reportar.")


          REPORTE - CASO 2 (CBC)
1. RESUMEN FINANCIERO
   ------------------------------------------------
   COSTO OPERATIVO REAL:      $415,525 COP
   ------------------------------------------------
   - Flota (Fijo):            $200,000 (4 vehículos)
   - Distancia Variable:      $163,175 (65.3 km)
   - Tiempo:                  $19,842
   - Combustible:             $32,508

2. ESTADO DE LOS DEPÓSITOS
   ------------------------------------------------
   Depósito   | Capacidad  | Uso        | Estado         
   CD05       | 28         | 26         | 🟢 OK           
   CD09       | 43         | 42         | 🟢 OK           
   CD11       | 16         | 15         | 🟢 OK           
   CD12       | 18         | 18         | 🟢 OK           


In [11]:
# ==========================================
# CELDA 6: MAPA INTERACTIVO
# ==========================================
import folium

# Verificar si hay datos para graficar
if 'df_out' in locals() and not df_out.empty:
    print("Iniciando generación del mapa...")

    # 1. Configurar Mapa Base (Centrado usando clients_data)
    # Usamos la lista clients_data que definimos en la Celda 2
    try:
        avg_lat = sum(c['lat'] for c in clients_data) / len(clients_data)
        avg_lon = sum(c['lon'] for c in clients_data) / len(clients_data)
    except:
        avg_lat, avg_lon = 4.65, -74.1 # Fallback a Bogotá centro si falla el cálculo

    mapa = folium.Map([avg_lat, avg_lon], zoom_start=12, tiles='CartoDB positron')

    # 2. Colores
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue', 'black']

    # 3. Dibujar Depósitos
    print("   - Dibujando depósitos...")
    for d in depot_ids:
        node = nodes[d] # Buscar en el diccionario de nodos

        # Verificar holgura
        slack_val = 0
        if hasattr(m, 'slack_depot'):
            slack_val = pyo.value(m.slack_depot[d])

        popup_txt = f"{d}<br>Cap: {node['cap']}"
        icon_color = 'black'

        if slack_val > 0.1:
            popup_txt += f"<br>SOBRECUPO: {slack_val:.1f}"
            icon_color = 'red'

        folium.Marker(
            [node['lat'], node['lon']],
            popup=popup_txt,
            icon=folium.Icon(color=icon_color, icon='warehouse', prefix='fa')
        ).add_to(mapa)

    # 4. Dibujar Clientes
    print("   - Dibujando clientes...")
    for c in client_ids:
        node = nodes[c]
        folium.CircleMarker(
            [node['lat'], node['lon']],
            radius=5, color='gray', fill=True,
            popup=f"{c} ({node['dem']}kg)"
        ).add_to(mapa)

    # 5. Dibujar Rutas (CON DIAGNÓSTICO)
    print("   - Dibujando rutas...")
    count_routes = 0

    for _, r in df_out.iterrows():
        # Obtener secuencia y limpiar espacios por si acaso
        seq_str = str(r['RouteSequence']).strip()
        node_list = seq_str.split('-')

        # Construir lista de coordenadas
        pts = []
        for n in node_list:
            n_clean = n.strip()
            if n_clean in nodes:
                pts.append([nodes[n_clean]['lat'], nodes[n_clean]['lon']])
            else:
                print(f"Advertencia: Nodo {n_clean} no encontrado en coordenadas.")

        if len(pts) > 1:
            # Elegir color basado en el depósito
            try:
                color_idx = depot_ids.index(r['DepotId']) % len(colors)
                color = colors[color_idx]
            except:
                color = 'blue'

            folium.PolyLine(
                pts,
                color=color,
                weight=3,
                opacity=0.8,
                tooltip=f"{r['VehicleId']} ({r['DepotId']})"
            ).add_to(mapa)
            count_routes += 1

    print(f" Se dibujaron {count_routes} rutas exitosamente.")

    # 6. Guardar y Mostrar
    filename = "mapa_final_cbc.html"
    mapa.save(filename)
    print(f" Archivo guardado: {filename}")

    try:
        from IPython.display import display
        display(mapa)
    except:
        print("Abre el archivo HTML para ver el mapa.")

else:
    print(" Error: No se encontró el DataFrame 'df_out' con los resultados.")

Iniciando generación del mapa...
   - Dibujando depósitos...
   - Dibujando clientes...
   - Dibujando rutas...
 Se dibujaron 4 rutas exitosamente.
 Archivo guardado: mapa_final_cbc.html


# Análisis de Resultados y Conclusiones Finales (Caso 2)

### Desempeño del Modelo
La implementación robusta en **Pyomo** utilizando el solver **CBC** fue exitosa, logrando converger a una **Solución Óptima Global** en el tiempo asignado.
* **Estado Final:** `Optimal` (Gap 0.00%).
* **Validación:** Se cumplieron el 100% de las demandas de los clientes sin violar restricciones duras de capacidad, evitando el pago de penalizaciones.

### Análisis Financiero
El costo operativo total minimizado es de $415,525 COP. La estructura de costos revela la naturaleza de la operación:

1. Costos Fijos (48%): $200,000 COP. Representa el mayor componente,
 derivado de la activación de 4 vehículos. Esto sugiere que la reducción de flota es la palanca más fuerte para ahorro futuro.

2. Costos Variables (Distancia + Tiempo + Combustible): ~$215,000 COP. La optimización de rutas logró comprimir el recorrido total a solo **65.3 km** para atender toda la demanda, una cifra altamente eficiente para la dispersión geográfica de los clientes.

### Gestión de Infraestructura y Cuellos de Botella
El análisis de uso de los Centros de Distribución (CDs) arroja hallazgos estratégicos críticos:

| Depósito | Capacidad | Uso Real | % Ocupación | Diagnóstico |
| :--- | :--- | :--- | :--- | :--- |
| **CD12** | 18 kg | 18 kg | **100%** | 🔴 **Cuello de Botella Crítico** |
| **CD09** | 43 kg | 42 kg | **98%** | 🟡 **Saturado** |
| **CD05** | 28 kg | 26 kg | **93%** | 🟢 Eficiente |
| **CD11** | 16 kg | 15 kg | **94%** | 🟢 Eficiente |
| **CD06** | 3 kg | 0 kg | 0% | ⚫ **Inviable / No Utilizado** |

**Insights Clave:**
1.  **Saturación Crítica:** La operación está al límite en **CD12** y **CD09**. Un aumento mínimo en la demanda de sus zonas asignadas haría el sistema infactible (o requeriría penalizaciones).
2.  **Inteligencia del Solver:** El modelo decidió **no utilizar el depósito CD06** (Capacidad: 3kg). Al detectar que su capacidad era insuficiente para las demandas cercanas (que rondan los 10-15kg), el algoritmo prefirió rutear vehículos desde depósitos más lejanos pero con capacidad (como CD05 o CD09), evitando así el sobrecosto por "multa" de capacidad.

### Conclusión y Recomendaciones
El modelo actual es **operativamente factible y matemáticamente óptimo**. Para mejorar la robustez de la cadena de suministro ante variaciones futuras de demanda, se recomienda:
1.  **Ampliar la capacidad del CD12:** Es el punto más frágil de la red.
2.  **Cerrar o Reestructurar CD06:** Su capacidad actual de 3kg es irrelevante para la magnitud de los pedidos (promedio 11kg/cliente), haciéndolo inútil en la práctica logística actual.

## Caso 3: Modelo Logístico Realista con Múltiples Centros y Vehículos Heterogéneos

### Evolución del Modelo

Este escenario representa la versión más completa y cercana a una **operación logística urbana real**.  
A diferencia de los Casos 1 y 2, aquí convergen simultáneamente tres dimensiones clave:

- **Múltiples Centros de Distribución (CDs)** con capacidades y ubicaciones diferenciadas.  
- **Vehículos heterogéneos**, cada uno con capacidad, costos y desempeño propios.  
- **Restricciones urbanas y operacionales**, incluyendo zonas exclusivas por tipo de vehículo, horarios de operación y limitaciones de acceso.

El desafío ya no consiste únicamente en trazar rutas eficientes, sino en lograr una **coordinación integral del sistema logístico**:  
determinar qué vehículo parte de qué centro, qué clientes puede atender según las restricciones urbanas y cómo **minimizar el costo logístico total** en un entorno complejo y dinámico.

### Enfoque de Resolución

Para manejar simultáneamente la escala del problema (50–100 clientes) y las restricciones heterogéneas, se adopta un **enfoque híbrido** que combina modelación matemática rigurosa con algoritmos heurísticos avanzados.

#### 1. Modelación Base con Pyomo
Se definen formalmente:
- los **conjuntos** (clientes, vehículos, depósitos),
- los **parámetros** (capacidades, demandas, costos, distancias),
- y las **restricciones** principales.

Esto garantiza la **correspondencia exacta** entre la formulación matemática (Etapa 1) y su implementación computacional.

#### 2. Resolución del VRP mediante OR-Tools
Debido al crecimiento cuadrático del número de arcos, un solver **MILP exacto** deja de ser escalable para este caso.  
OR-Tools permite:

- obtener soluciones **viables y de buena calidad**,  
- en tiempos computacionales razonables,  
- manteniendo la posibilidad de incorporar restricciones complejas del entorno urbano.

Su uso constituye un **trade-off** entre optimalidad teórica y viabilidad práctica.

#### 3. Restricciones Urbanas por Tipo de Vehículo
La **matriz de costos** se modifica introduciendo penalizaciones prohibitivas para impedir que un vehículo:

- ingrese a zonas donde no está autorizado,
- exceda horarios,
- o viole reglas de circulación urbana.

Este mecanismo garantiza la coherencia entre la operación real y la solución calculada.

#### 4. Capacidad y Posible Retorno al Centro
El modelo contempla que un vehículo pueda **regresar a su centro** para reabastecerse o descargar, lo cual refleja mejor la operación real y evita rutas físicamente inviables.

Este enfoque híbrido permite equilibrar:

- **factibilidad computacional**,  
- **realismo operativo**,  
- y **calidad de la solución**,

logrando un diseño robusto para apoyar decisiones empresariales en contextos logísticos urbanos complejos.


In [12]:
!pip install ortools folium pyomo

In [13]:

#  Configuración de rutas locales


# Solo cambiamos las rutas de entrada/salida
CLIENTS_CSV  = "datos_caso3/clients.csv"
DEPOTS_CSV   = "datos_caso3/depots.csv"
VEHICLES_CSV = "datos_caso3/vehicles.csv"
PARAM_CSV    = "datos_caso3/parameters_urban.csv"

OUTPUT_VERIF = "datos_caso3/verificacion_caso3.csv"
OUTPUT_MAP   = "datos_caso3/map_caso3.html"

print("Archivos de entrada:")
print(" -", CLIENTS_CSV)
print(" -", DEPOTS_CSV)
print(" -", VEHICLES_CSV)
print(" -", PARAM_CSV)
print("Archivos de salida:")
print(" -", OUTPUT_VERIF)
print(" -", OUTPUT_MAP)


Archivos de entrada:
 - datos_caso3/clients.csv
 - datos_caso3/depots.csv
 - datos_caso3/vehicles.csv
 - datos_caso3/parameters_urban.csv
Archivos de salida:
 - datos_caso3/verificacion_caso3.csv
 - datos_caso3/map_caso3.html


In [14]:

import pandas as pd
import math, csv, os
import folium

# OR-Tools
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Pyomo mínimo (solo conjuntos y parámetros)
from pyomo.environ import ConcreteModel, Set, Param


# ---------------------------
# Distancia Haversine
# ---------------------------
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = (math.sin(dphi/2)**2 +
         math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2)
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))


# ---------------------------
# Lectura de datos
# ---------------------------
clients  = pd.read_csv(CLIENTS_CSV)
depots   = pd.read_csv(DEPOTS_CSV)
vehicles = pd.read_csv(VEHICLES_CSV)
params   = pd.read_csv(PARAM_CSV)

clients["id"] = clients["StandardizedID"]
clients["lat"] = clients["Latitude"]
clients["lon"] = clients["Longitude"]
clients["demand"] = clients["Demand"]
clients["restriction"] = clients.get("VehicleSizeRestriction", None)

depots["id"] = depots["StandardizedID"]
depots["lat"] = depots["Latitude"]
depots["lon"] = depots["Longitude"]

vehicles["id"] = vehicles["StandardizedID"]
vehicles["capacity"] = vehicles["Capacity"]
vehicles["vtype"] = vehicles.get("VehicleType", "")


# ---------------------------
# Asignación de depósitos a vehículos
# ---------------------------
if "DepotID" in vehicles.columns:
    vehicles["start_depot"] = vehicles["DepotID"].astype(str)
    print("DepotID encontrado — se usa el archivo directamente.")
else:
    depot_ids = list(depots["id"])
    vehicles["start_depot"] = [ depot_ids[i % len(depot_ids)] for i in range(len(vehicles)) ]
    print("DepotID ausente — asignación automática round-robin.")


# ---------------------------
# Parámetros de costo
# ---------------------------
P = {row["Parameter"]: row["Value"] for _, row in params.iterrows()}
fuel_price = float(P.get("fuel_price", 16300))
fuel_eff   = float(P.get("fuel_efficiency_van_small_min", 30))
fixed_cost = float(P.get("C_fixed", 50000))
cost_km    = float(P.get("C_dist", 2500))
cost_hour  = float(P.get("C_time", 7600))
avg_speed  = 30.0


# ===============================================================
# PYOMO (mínimo) — Definición de conjuntos y parámetros solamente
# ===============================================================

model = ConcreteModel()

# conjunto de depósitos
model.D = Set(initialize=list(depots["id"]))

# conjunto de clientes
model.C = Set(initialize=list(clients["id"]))

# conjunto de vehículos
model.V = Set(initialize=list(vehicles["id"]))

# demanda del cliente
demand_dict = {row["id"]: float(row["demand"]) for _, row in clients.iterrows()}
model.d = Param(model.C, initialize=demand_dict)

# capacidad del vehículo
cap_dict = {row["id"]: float(row["capacity"]) for _, row in vehicles.iterrows()}
model.cap = Param(model.V, initialize=cap_dict)

print("Pyomo: conjuntos y parámetros cargados correctamente.")


# ===============================================================
# Construcción de nodos para OR-Tools
# ===============================================================

nodes = []
for _, d in depots.iterrows():
    nodes.append({"id": d["id"], "lat": float(d["lat"]), "lon": float(d["lon"]), "type": "depot"})

for _, c in clients.iterrows():
    nodes.append({
        "id": c["id"],
        "lat": float(c["lat"]),
        "lon": float(c["lon"]),
        "type": "client",
        "demand": float(c["demand"]),
        "restriction": c["restriction"]
    })

node_index = {n["id"]: i for i, n in enumerate(nodes)}
num_nodes = len(nodes)


# ---------------------------
# Matriz de distancias
# ---------------------------
dist_km = [[0]*num_nodes for _ in range(num_nodes)]
dist_m  = [[0]*num_nodes for _ in range(num_nodes)]

for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            d = haversine_km(nodes[i]["lat"], nodes[i]["lon"], nodes[j]["lat"], nodes[j]["lon"])
            dist_km[i][j] = d
            dist_m[i][j] = int(d * 1000)


# ===============================================================
# OR-TOOLS VRP
# ===============================================================

num_vehicles = len(vehicles)
starts = [ node_index[vehicles.iloc[v]["start_depot"]] for v in range(num_vehicles) ]
ends = starts[:]  # mismo depósito de retorno

manager = pywrapcp.RoutingIndexManager(num_nodes, num_vehicles, starts, ends)
routing = pywrapcp.RoutingModel(manager)


# ---------------------------
# Callback de distancia con restricciones por tamaño de vehículo
# ---------------------------
def make_dist_cb(v_idx):
    vtype = str(vehicles.iloc[v_idx]["vtype"]).strip().lower()
    def cb(from_index, to_index):
        fn = manager.IndexToNode(from_index)
        tn = manager.IndexToNode(to_index)
        if nodes[tn]["type"] == "client":
            need = nodes[tn]["restriction"]
            if need is not None and str(need).strip() != "":
                if str(need).strip().lower() != vtype:
                    return int(1e9)
        return dist_m[fn][tn]
    return cb

for v in range(num_vehicles):
    idx = routing.RegisterTransitCallback(make_dist_cb(v))
    routing.SetArcCostEvaluatorOfVehicle(idx, v)


# ---------------------------
# Capacidad
# ---------------------------
demands = [0]*num_nodes
for i,n in enumerate(nodes):
    if n["type"] == "client":
        demands[i] = int(n["demand"])

def demand_cb(index):
    return demands[ manager.IndexToNode(index) ]

demand_idx = routing.RegisterUnaryTransitCallback(demand_cb)
vehicle_caps = [int(c) for c in list(vehicles["capacity"])]
routing.AddDimensionWithVehicleCapacity(demand_idx, 0, vehicle_caps, True, "Capacity")


# ---------------------------
# Solución
# ---------------------------
params = pywrapcp.DefaultRoutingSearchParameters()
params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
params.time_limit.seconds = 30

solution = routing.SolveWithParameters(params)
print("Solución encontrada.")


# ===============================================================
# Archivo de verificación
# ===============================================================
with open(OUTPUT_VERIF, "w", newline='', encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["VehicleId","DepotId","InitialLoad","RouteSequence","ClientsServed",
                "DemandsSatisfied","TotalDistance","TotalTime","FuelCost"])

    for v in range(num_vehicles):
        start_idx = routing.Start(v)
        idx = start_idx
        route = []
        dem_list = []
        total_dist = 0
        total_time = 0
        clients_served = 0
        load = 0

        while not routing.IsEnd(idx):
            n = manager.IndexToNode(idx)
            route.append(nodes[n]["id"])

            nxt = solution.Value(routing.NextVar(idx))
            nxt_node = manager.IndexToNode(nxt)
            total_dist += dist_km[n][nxt_node]

            if nodes[n]["type"] == "client":
                d = nodes[n]["demand"]
                dem_list.append(int(d))
                load += d
                clients_served += 1

            total_time += (dist_km[n][nxt_node] / avg_speed) * 60
            idx = nxt

        n = manager.IndexToNode(idx)
        route.append(nodes[n]["id"])

        gallons = total_dist / fuel_eff
        fuel_cost = gallons * fuel_price

        if clients_served == 0:
            w.writerow([vehicles.iloc[v]["id"], vehicles.iloc[v]["start_depot"], 0,
                        vehicles.iloc[v]["start_depot"]+"-"+vehicles.iloc[v]["start_depot"],
                        0,"",0,0,0])
        else:
            w.writerow([
                vehicles.iloc[v]["id"],
                vehicles.iloc[v]["start_depot"],
                round(load,2),
                "-".join(route),
                clients_served,
                "-".join(str(x) for x in dem_list),
                round(total_dist,3),
                round(total_time,2),
                round(fuel_cost,2)
            ])

print("Archivo verificacion_caso3.csv escrito:", OUTPUT_VERIF)

# Mapa Folium
m = folium.Map(location=[4.70, -74.10], zoom_start=11)

for _, d in depots.iterrows():
    folium.Marker([d["lat"], d["lon"]], popup=f"Depósito: {d['id']}",
                  icon=folium.Icon(color="red")).add_to(m)

for _, c in clients.iterrows():
    folium.CircleMarker([c["lat"], c["lon"]], radius=4,
                         popup=f"Cliente: {c['id']}<br>Demanda: {c['demand']}",
                         color="blue", fill=True).add_to(m)

colors = ["red","blue","green","purple","orange","black","darkgreen","cadetblue","pink"]

for v in range(num_vehicles):
    idx = routing.Start(v)
    coords = []
    while not routing.IsEnd(idx):
        n = manager.IndexToNode(idx)
        coords.append((nodes[n]["lat"], nodes[n]["lon"]))
        idx = solution.Value(routing.NextVar(idx))
    n = manager.IndexToNode(idx)
    coords.append((nodes[n]["lat"], nodes[n]["lon"]))
    if len(coords) > 2:
        folium.PolyLine(coords, color=colors[v % len(colors)], weight=3).add_to(m)

m.save(OUTPUT_MAP)
print("Mapa generado:", OUTPUT_MAP)


DepotID ausente — asignación automática round-robin.
Pyomo: conjuntos y parámetros cargados correctamente.
Solución encontrada.
Archivo verificacion_caso3.csv escrito: datos_caso3/verificacion_caso3.csv
Mapa generado: datos_caso3/map_caso3.html


In [15]:
display(m)

In [16]:
import pandas as pd
pd.read_csv(OUTPUT_VERIF).head()

,VehicleId,DepotId,InitialLoad,RouteSequence,ClientsServed,DemandsSatisfied,TotalDistance,TotalTime,FuelCost
0,V001,CD01,121.0,CD01-C004-C047-C002-C063-C081-C082-C012-C020-C...,9,22-12-15-12-12-12-12-12-12,25.447,50.89,11850.92
1,V002,CD02,0.0,CD02-CD02,0,NaN,0.000,0.00,0.00
2,V003,CD03,0.0,CD03-CD03,0,NaN,0.000,0.00,0.00
3,V004,CD04,9.0,CD04-C007-CD04,1,9,0.666,1.33,310.01
4,V005,CD05,0.0,CD05-CD05,0,NaN,0.000,0.00,0.00


### Análisis Estratégico del Caso 3

#### 1. Parámetros Críticos que Afectan la Operación

Del análisis de sensibilidad se identifican tres parámetros con **impacto dominante** sobre la operación logística:

- **Costo de combustible**: influye directamente en el costo total y favorece soluciones con rutas más cortas y eficientes.  
- **Demandas de los clientes**: determinan el nivel de saturación del sistema y condicionan el número de vehículos necesarios.  
- **Capacidad de los Centros de Distribución (CDs)**: modifica la congestión operativa, la longitud de las rutas y la eficiencia global del sistema.

Estos parámetros redefinen la **estructura del VRP** y deben ser monitoreados continuamente en la planificación operativa diaria.


#### 2. Cuellos de Botella Operacionales

El análisis revela tres fuentes principales de congestión:

- **Centros de distribución saturados**, con más demanda asignada de la que su infraestructura puede manejar.  
- **Zonas urbanas restringidas**, donde únicamente pueden operar vehículos pequeños.  
- **Clientes geográficamente dispersos**, que generan rutas largas y tiempos de operación elevados.

Detectar estos cuellos de botella permite **priorizar inversiones**, rediseñar rutas y optimizar el uso de la flota.


#### 3. Impacto de las Variaciones de Parámetros

- **Costo de combustible (±20 %)**: modifica significativamente el costo total de operación; el impacto es mayor en vehículos con baja eficiencia energética.  
- **Capacidad de los centros**: aumentar la capacidad reduce rutas redundantes y mejora el desempeño logístico general.  
- **Demandas de los clientes**: incrementos fuertes generan necesidades adicionales de reabastecimiento y mayores requerimientos de flota.

Estos factores muestran que el sistema es **sensible tanto a aspectos operativos como energéticos**.


#### 4. Recomendaciones para LogistiCo

A partir de los resultados, se sugieren las siguientes acciones estratégicas:

- **Incrementar la capacidad** o mejorar la distribución de carga entre los centros existentes.  
- **Refinar la flota**, fortaleciendo la disponibilidad de vehículos pequeños en zonas con restricciones urbanas.  
- **Incorporar modelos predictivos** que anticipen fluctuaciones en la demanda y ajusten la operación.  
- **Monitorear continuamente el costo del combustible** para adaptar las políticas de asignación y despliegue vehicular.

Estas medidas contribuyen a una operación más estable, flexible y resiliente.

### 5. Preguntas Estratégicas del Caso

**¿Cuáles parámetros influyen más en la logística urbana?**  
Principalmente el **costo del combustible**, las **demandas** y la **capacidad** de los centros.

**¿Dónde se presentan los mayores cuellos de botella?**  
En **centros saturados** y en **zonas con restricciones operativas**.

**¿Qué mejoras se recomiendan?**  
Una **flota más diversificada**, una **redistribución geográfica de carga** y una **expansión selectiva de centros críticos**.
